 # <center><font color='red'> TME 1 - Régression linéaire et descente de gradient </font></center>

In [7]:
# Ne pas oublier d'executer dans le shell avant de lancer python :
# source /users/Enseignants/piwowarski/venv/amal/3.7/bin/activate

import timeit
import torch
from torch.autograd import Function
from torch.autograd import gradcheck
# from datamaestro import prepare_dataset 
from sklearn.datasets import load_boston
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

In [3]:
class Context:
    """Very simplified context object"""
    def __init__(self):
        self._saved_tensors = ()
    def save_for_backward(self, *args):
        self._saved_tensors = args
    @property
    def saved_tensors(self):
        return self._saved_tensors


# class MaFonction(Function):
#     @staticmethod
#     def forward(ctx,input):
#         ctx.save_for_backward(input)
#         return None

#     @staticmethod
#     def backward(ctx,grad_output):
#         input = ctx.saved_tensors
#         return None


## Exemple d'implementation de fonction a 2 entrÃ©es
class MaFonction(Function):
    @staticmethod
    def forward(ctx,x,w):
        ## Calcul la sortie du module
        ctx.save_for_backward(x,w)
        return torch.mm(x, w.T)

    @staticmethod
    def backward(ctx, forward_output):
        ## Calcul du gradient du module par rapport a chaque groupe d'entrÃ©es
        x,w = ctx.saved_tensors
        return torch.mm( forward_output, w), torch.mm( forward_output.T , x )

    @staticmethod
    def mse(yhat, y):
        return ((yhat-y)**2).mean()
    
    @staticmethod
    def sgb(ctx, grad_output, eps):
        x,w = ctx.saved_tensors
        w = w - eps * grad_output[1]
        return w

# <font color="blue"> 1. Préliminaire - Données random </font>

In [4]:
x = torch.randn(10,5,requires_grad=True,dtype=torch.float64)
w = torch.randn(1,5,requires_grad=True,dtype=torch.float64)

## Pour utiliser la fonction 
mafonction = MaFonction()
ctx = Context()
forward = mafonction.forward(ctx,x,w)
backward = mafonction.backward(ctx, forward)

## Pour tester le gradient 
mafonction_check = MaFonction.apply
torch.autograd.gradcheck(mafonction_check,(x,w))

# ## Pour telecharger le dataset Boston
# ds=prepare_dataset("edu.uci.boston")
# fields, data =ds.files.data() 

True

# <font color="blue"> 2. Application des descentes de gradient sur les données Boston </font>

## Chargement des données Boston

In [24]:
X, y = load_boston(return_X_y=True) # (506, 13)  (506,)
X = torch.from_numpy(X)
y = torch.from_numpy(y)

print(X.shape, y.shape)

torch.Size([506, 13]) torch.Size([506])


In [25]:
class BostonDataset(Dataset):
    
    def __init__(self, X, y):
        super(BostonDataset, self).__init__()
        self.data = X
        self.y = y
        
    def __getitem__(self, index):
        return self.data[index], self.y[index]
    
    def __len__(self):
        return len(self.y)
        
dataset = BostonDataset(X, y)

train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

print(len(train_dataset), len(test_dataset))

train_dataloader = DataLoader(train_dataset, batch_size=50, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=50, shuffle=True)

404 102


## SGB

In [21]:
epoch = 200
loss_train = []
loss_test = []

w = torch.randn(1, 13, requires_grad=True, dtype=torch.float64)
mafonction = MaFonction()
ctx = Context()

print("Training phase")
for _ in tqdm(range(epoch)):
    loss = 0
    c = 0
    for _, (data, label) in enumerate(train_dataloader):  
        forw = mafonction.forward(ctx, data, w)
        backw = mafonction.backward(ctx, forw)
        mse = mafonction.mse(forw, label)
        loss += mse.item()
        w = mafonction.sgb(ctx, backw, 1e-5)
        c += 1
    loss_train.append(loss / c)
    
print("Testing phase")
for _ in tqdm(range(epoch)):
    loss = 0
    c = 0
    for _, (data, label) in enumerate(test_dataloader):
        forw = mafonction.forward(ctx, data, w)
        mse = mafonction.mse(forw, label)
        loss += mse.item()
        c += 1
    loss_test.append(loss / c)
    

 10%|▉         | 19/200 [00:00<00:00, 183.12it/s]

Training phase


 41%|████      | 82/200 [00:00<00:00, 813.98it/s]

Testing phase


100%|██████████| 200/200 [00:00<00:00, 799.06it/s]


In [26]:
len(loss_train)

200

In [30]:
def sgbBoston(data, labels, pc_train, version, eps=1e-5):
    size_split = int(pc_train*len(data))
    X_train, X_test = data[:size_split], data[size_split:]
    y_train, y_test = labels[:size_split], labels[size_split:]
    w = torch.randn(1, 13, requires_grad=True, dtype=torch.float64)
    mafonction = MaFonction()
    ctx = Context()
    mse_train = []
    mse_test = []
    writer = SummaryWriter()
    
    tic=timeit.default_timer()
    
    if version == "batch":
        for i in range(200):
            for j in range(50):
                forward = mafonction.forward(ctx, X_train, w)
                backward = mafonction.backward(ctx, forward)
                mse = mafonction.mse(forward, y_train)
                #mse_train.append(mse)
                writer.add_scalar('MSE/batch/train', mse, j)
                w = mafonction.sgb(ctx, backward, eps)
            forward = mafonction.forward(ctx, X_test, w)
            mse = mafonction.mse(forward, y_test)
            #mse_test.append(mse)
            writer.add_scalar('MSE/batch/test', mse, i)
            
    elif version == "stochastique":
        for i in range(200):
            for j in range(50):
                rand = np.random.randint(0,len(X_train))
                forward = mafonction.forward(ctx, X_train[rand].reshape((1,13)), w)
                backward = mafonction.backward(ctx, forward)
                mse = mafonction.mse(forward, y_train[rand])
                #mse_train.append(mse)
                writer.add_scalar('MSE/stoch/train', mse, j)
                w = mafonction.sgb(ctx, backward, eps)
            forward = mafonction.forward(ctx, X_test, w)
            mse = mafonction.mse(forward, y_test)
            #mse_test.append(mse)
            writer.add_scalar('MSE/stoch/test', mse, i)
            
    elif version == "minibatch":
        for i in range(200):
            for j in range(0, len(X_train)-10, 10):
                forward = mafonction.forward(ctx, X_train[j:j+10], w)
                backward = mafonction.backward(ctx, forward)
                mse = mafonction.mse(forward, y_train[j:j+10])
                #mse_train.append(mse)
                writer.add_scalar('MSE/minib/train', mse, j)
                w = mafonction.sgb(ctx, backward, eps)
            forward = mafonction.forward(ctx, X_test, w)
            mse = mafonction.mse(forward, y_test)
            #mse_test.append(mse)
            writer.add_scalar('MSE/minib/test', mse, i)
            
    else:
        print("Version inexistante, essayez : minibatch | batch | stochastique")
        
    toc=timeit.default_timer()
    
    print("Variant : ", version)
    print("Learning rate : ", eps)
    print("TIME : ", toc-tic)
    print("\n")
    
    return None

In [29]:
versions = ["batch", "stochastique", "minibatch"]

# Décommentez le bloc suivant pour calculer la descente de gradient des 3 versions
# for v in versions:
#     sgbBoston(X, y, 0.8, v, 1e-10)

Variant :  batch
Learning rate :  1e-10
TIME :  19.63852430000003
Variant :  stochastique
Learning rate :  1e-10
TIME :  5.9461709999999925
Variant :  minibatch
Learning rate :  1e-10
TIME :  4.064470300000039


<font color="green"> Pour pouvoir afficher les résultats : 
    <ul>
        <li> Ouvrir l'anaconda prompt </li>
        <li> Se placer dans le dossier contenant le sous-dossier "runs" </li>
        <li> Taper la commande : tensorboard --logdir=runs </li>
        <li> Taper sur google le lien localhost affiché </li>
    </ul>
</font>

![MSE](mse_graphes.jpg)

<font color="green"> 
Voici les résultats qu'on a obtenu ci-dessus avec les différentes variantes de la descente de gradient et avec un learning rate de 1e-10. <br>
    
Dans un premier temps, on remarque que la vitesse de convergence de la variante batch est 4 fois plus lente que les deux autres variantes (20s vs 5s), ce qui n'est pas surprenant vu qu'on prend en compte l'ensemble de nos données pour l'apprentissage cependant cette variante comporte divers avantages comme par exemple un taux d'apprentissage qui ne décroit pas, la garantie de converger à un minimum global si la fonction loss est convexe puis l'estimation du gradient se retrouve non biaisé. <br>
    
Concernant la variante mini-batch, on observe sur la courbe de train plusieurs oscillations représentant le bruit ajouté puisque la taille du batch est inférieure à celle de l'ensemble du train, à cause de ce bruit, il n'est pas possible de converger sans faire décroitre le learning rate. <br>
    
Concernant la variante stochastique, elle présente encore plus de bruit que la variante minibatch et également un temps d'exécution plus long. <br>
</font>